# VirtualiZarr + Cubed example

In [1]:
import xarray as xr

# create an example pre-existing netCDF4 file
ds = xr.tutorial.open_dataset('air_temperature')
ds

<xarray.Dataset> Size: 31MB
Dimensions:  (lat: 25, time: 2920, lon: 53)
Coordinates:
  * lat      (lat) float32 100B 75.0 72.5 70.0 67.5 65.0 ... 22.5 20.0 17.5 15.0
  * lon      (lon) float32 212B 200.0 202.5 205.0 207.5 ... 325.0 327.5 330.0
  * time     (time) datetime64[ns] 23kB 2013-01-01 ... 2014-12-31T18:00:00
Data variables:
    air      (time, lat, lon) float64 31MB ...
Attributes:
    Conventions:  COARDS
    title:        4x daily NMC reanalysis (1948)
    description:  Data is from NMC initialized reanalysis\n(4x/day).  These a...
    platform:     Model
    references:   http://www.esrl.noaa.gov/psd/data/gridded/data.ncep.reanaly...

In [2]:
ds.to_netcdf('air.nc')

/var/folders/8t/gslp67x10vgfjgv68qj92n0m0000gn/T/ipykernel_95257/3753223073.py:1: SerializationWarning: saving variable air with floating point data as an integer dtype without any _FillValue to use for NaNs
  ds.to_netcdf('air.nc')


In [3]:
#from virtualizarr.readers.hdf import HDFVirtualBackend

In [4]:
from virtualizarr import open_virtual_dataset

# vds = open_virtual_dataset('air.nc', backend=HDFVirtualBackend)
vds = open_virtual_dataset('air.nc')
vds

/Users/tom/miniforge3/envs/cubed-virtualizarr-example/lib/python3.11/site-packages/virtualizarr/readers/common.py:58: UserWarning: Specifying `indexes=None` will create in-memory pandas indexes for each 1D coordinate, but concatenation of ManifestArrays backed by pandas indexes is not yet supported (see issue #18).You almost certainly want to pass `indexes={}` to `open_virtual_dataset` instead.
  warnings.warn(


<xarray.Dataset> Size: 8MB
Dimensions:  (time: 2920, lat: 25, lon: 53)
Coordinates:
  * time     (time) float32 12kB ManifestArray<shape=(2920,), dtype=float32, ...
  * lon      (lon) float32 212B ManifestArray<shape=(53,), dtype=float32, chu...
  * lat      (lat) float32 100B ManifestArray<shape=(25,), dtype=float32, chu...
Data variables:
    air      (time, lat, lon) int16 8MB ManifestArray<shape=(2920, 25, 53), d...
Attributes:
    Conventions:  COARDS
    title:        4x daily NMC reanalysis (1948)
    description:  Data is from NMC initialized reanalysis\n(4x/day).  These a...
    platform:     Model
    references:   http://www.esrl.noaa.gov/psd/data/gridded/data.ncep.reanaly...

In [5]:
marr = vds['air'].data
marr

ManifestArray<shape=(2920, 25, 53), dtype=int16, chunks=(730, 13, 27)>

In [6]:
marr.manifest

ChunkManifest<shape=(1, 1, 1)>

In [7]:
marr.zarray

ZArray(shape=(2920, 25, 53), chunks=(730, 13, 27), dtype=dtype('int16'), fill_value=0, order='C', compressor=None, filters=[], zarr_format=2)

In [8]:
import numpy as np

concatenated = np.concatenate([marr, marr], axis=0)
concatenated

ManifestArray<shape=(5840, 25, 53), dtype=int16, chunks=(730, 13, 27)>

In [9]:
concatenated.manifest.dict()

{'0.0.0': {'path': 'file:///Users/tom/workspace/cubed/air.nc',
  'offset': 15419,
  'length': 7738000},
 '1.0.0': {'path': 'file:///Users/tom/workspace/cubed/air.nc',
  'offset': 15419,
  'length': 7738000}}

In [10]:
ds1 = ds.isel(time=slice(None, 1460))
ds2 = ds.isel(time=slice(1460, None))

ds1.to_netcdf('air1.nc')
ds2.to_netcdf('air2.nc')

/var/folders/8t/gslp67x10vgfjgv68qj92n0m0000gn/T/ipykernel_95257/3294039818.py:4: SerializationWarning: saving variable air with floating point data as an integer dtype without any _FillValue to use for NaNs
  ds1.to_netcdf('air1.nc')
/var/folders/8t/gslp67x10vgfjgv68qj92n0m0000gn/T/ipykernel_95257/3294039818.py:5: SerializationWarning: saving variable air with floating point data as an integer dtype without any _FillValue to use for NaNs
  ds2.to_netcdf('air2.nc')


In [13]:
vds1 = open_virtual_dataset('air1.nc', indexes={})
vds2 = open_virtual_dataset('air2.nc', indexes={})
# vds1 = open_virtual_dataset('air1.nc', indexes={}, backend=HDFVirtualBackend)
# vds2 = open_virtual_dataset('air2.nc', indexes={}, backend=HDFVirtualBackend)

In [14]:
combined_vds = xr.concat([vds1, vds2], dim='time', coords='minimal', compat='override')
combined_vds

<xarray.Dataset> Size: 8MB
Dimensions:  (time: 2920, lat: 25, lon: 53)
Coordinates:
    time     (time) float32 12kB ManifestArray<shape=(2920,), dtype=float32, ...
    lon      (lon) float32 212B ManifestArray<shape=(53,), dtype=float32, chu...
    lat      (lat) float32 100B ManifestArray<shape=(25,), dtype=float32, chu...
Data variables:
    air      (time, lat, lon) int16 8MB ManifestArray<shape=(2920, 25, 53), d...
Attributes:
    Conventions:  COARDS
    title:        4x daily NMC reanalysis (1948)
    description:  Data is from NMC initialized reanalysis\n(4x/day).  These a...
    platform:     Model
    references:   http://www.esrl.noaa.gov/psd/data/gridded/data.ncep.reanaly...

In [16]:
# create an icechunk store
import icechunk
from icechunk import IcechunkStore, StorageConfig, StoreConfig, VirtualRefConfig
storage = StorageConfig.filesystem(str('combined'))
store = IcechunkStore.create(storage=storage, mode="w", config=icechunk.StoreConfig(inline_chunk_threshold_bytes=1))
combined_vds.virtualize.to_icechunk(store)

In [18]:
store.commit("combined files")

'16FNDXWXGJWEYS3H2JJG'

In [ ]:
! pip install icechunk -U 'zarr==3.0.0b2'

In [29]:
! pip list | grep ice

icechunk                  0.1.0a7
